# POD Reference Update

In [22]:
import os
import pandas as pd

# Paths
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\POD-Reference.xlsx"
network_path1 = r"\\Quickbook2024\d\Drive D\QuickBooks\2- Year 2024\Purchase Order (Vendor)- POD"
network_path2 = r"\\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD"

# Gather all PDF file names
pdf_files = []

def gather_pdfs_from_path(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_files.append(file)

gather_pdfs_from_path(network_path1)
gather_pdfs_from_path(network_path2)

# Create DataFrame with POD and Reference
df = pd.DataFrame(pdf_files, columns=["PDF File Names"])
df['POD'] = df['PDF File Names'].apply(lambda x: x.split('_')[0])
df['Reference'] = df['PDF File Names'].apply(lambda x: '_'.join(x.split('_')[1:]).replace('.pdf', ''))
df_reorganized = df[['POD', 'Reference']]

# Write to external Excel file (safe for VLOOKUP/XLOOKUP)
df_reorganized.to_excel(output_path, index=False)
print("✅ POD-Reference.xlsx has been updated.")




✅ POD-Reference.xlsx has been updated.


# Print POD

In [25]:
import os
import re
import win32api
import win32print
from datetime import datetime

# Printing Function
def print_pdf(path):
    try:
        win32api.ShellExecute(0, "print", path, None, ".", 0)
        print(f"✅ Sent to printer: {os.path.basename(path)}")
    except Exception as e:
        print(f"❌ Failed to print {path}: {e}")

# ───────────────────────────────
# 1.  Folder with POD PDFs
# ───────────────────────────────
FOLDER = r"\\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD"

# ───────────────────────────────
# 2.  Copy-&-paste POD list here
# ───────────────────────────────
raw_input = """
POD-251091  
"""

# ───────────────────────────────
# 3.  Parse & numerically sort PODs
# ───────────────────────────────
pod_list = [p.strip() for p in raw_input.replace("\n", "").split(",") if p.strip()]
PODS_ORDERED = sorted(pod_list, key=lambda x: int(x.split("-")[1]))  # ascending 250382 → 250443 …
PODS_SET     = set(PODS_ORDERED)   # fast membership test


## Only Print latest Version 

In [ ]:
# Track latest file per POD
latest_signed_files = {}

for pod in PODS_SET:
    latest_time = None
    latest_file = None

    for filename in os.listdir(FOLDER):
        if pod in filename:
            full_path = os.path.join(FOLDER, filename)
            modified_time = os.path.getmtime(full_path)

            if latest_time is None or modified_time > latest_time:
                latest_time = modified_time
                latest_file = full_path

    if latest_file:
            latest_signed_files[pod] = latest_file
            print(f"✅ Latest for {pod}: {latest_file} (Modified: {datetime.fromtimestamp(latest_time)})")

            print_pdf(latest_file) # Printing
    else:
        print(f"❌ No signed version found for {pod}")

✅ Latest for POD-251091: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD\POD-251091_NTA_SHI International_SO-20251079.pdf (Modified: 2025-07-30 23:11:05.267915)


## Only print signed Version

In [ ]:
# ───────────────────────────────
# 4.  Index every *_signed.pdf
#     Keep separate “revised” vs “regular” lists
# ───────────────────────────────
candidates = {pod: {"revised": [], "regular": []} for pod in PODS_SET}

for fname in os.listdir(FOLDER):
    if not fname.endswith("_signed.pdf"):
        continue
    for pod in PODS_SET:
        if pod in fname:
            base = fname.split("_")[0]             # e.g. POD-250382r2
            is_rev = "r" in base[len(pod):].lower()  # anything after the POD that contains 'r'
            key   = "revised" if is_rev else "regular"
            candidates[pod][key].append(fname)
            break
# ───────────────────────────────
# 5.  Helper: newest file in a list
# ───────────────────────────────
def newest(files):
    return max(files, key=lambda f: os.path.getmtime(os.path.join(FOLDER, f))) if files else None

# ───────────────────────────────
# 6.  Choose ONE file per POD
#     (prefer newest revised, else newest regular)
# ───────────────────────────────
chosen = {}
for pod, versions in candidates.items():
    chosen[pod] = newest(versions["revised"]) or newest(versions["regular"])
# ───────────────────────────────
# 7.  Print (or just list) in **numeric order**
# ───────────────────────────────

for pod in PODS_ORDERED:                 # keeps 250382 → … → 250864 order
    filename = chosen.get(pod)
    if filename:
        full_path = os.path.join(FOLDER, filename)
        print(full_path)                 # comment out after you verify
        print_pdf(full_path)           # ← uncomment to actually print